In [1]:
import pandas as pd
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas_datareader as pdr
import sqlite3
import threading

In [2]:
url='https://goodinfo.tw/StockInfo/index.asp'

In [3]:
def get_chrom(url,driver=r'C:\webdriver\chromedriver'):
    try:
        options=webdriver.ChromeOptions()
        options.add_argument('--headless')
        chrom=webdriver.Chrome(driver,options=options)
        chrom.implicitly_wait(3)
        chrom.get(url)
        return chrom
    except Exception as e:
        return None

In [4]:
xpath='/html/body/table[2]/tbody/tr/td[1]/div[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td/nobr/a'

In [5]:
chrom=get_chrom(url)
chrom.find_element(By.XPATH,xpath).click()

<ipython-input-3-8c400d98a754>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrom=webdriver.Chrome(driver,options=options)


In [6]:
xpath='/html/body/table[2]/tbody/tr/td[1]/div[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td/nobr/a'

In [7]:
chrom=get_chrom(url)
chrom.find_element(By.XPATH,xpath).click()

<ipython-input-3-8c400d98a754>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  chrom=webdriver.Chrome(driver,options=options)


In [8]:
xpath3='/html/body/table[2]/tbody/tr/td[3]/table/tbody/tr/td/div/form/nobr[3]/select/option[4]'

In [9]:
chrom.find_element(By.XPATH,xpath3).click()

In [10]:
soup=BeautifulSoup(chrom.page_source,'lxml')

In [11]:
test=soup.find('div',style="position:relative;width:811px;").find('tbody')

In [14]:
datas=[]
for i in test.find_all('tr',align="center"):
    #print(i.find('a',class_="link_black").text,i.find_all('td')[2].text,i.find_all('td')[4].text,i.find('td',align="right").text)
    datas.append([i.find('a',class_="link_black").text,i.find_all('td')[2].text])
    
datas

[['0050', '元大台灣50'],
 ['0050', '元大台灣50'],
 ['0052', '富邦科技'],
 ['0056', '元大高股息'],
 ['006203', '元大MSCI台灣'],
 ['006203', '元大MSCI台灣'],
 ['006204', '永豐臺灣加權'],
 ['006208', '富邦台50'],
 ['006208', '富邦台50'],
 ['00679B', '元大美債20年'],
 ['00679B', '元大美債20年'],
 ['00679B', '元大美債20年'],
 ['00679B', '元大美債20年'],
 ['00687B', '國泰20年美債'],
 ['00687B', '國泰20年美債'],
 ['00687B', '國泰20年美債'],
 ['00687B', '國泰20年美債'],
 ['00692', '富邦公司治理'],
 ['00692', '富邦公司治理'],
 ['00694B', '富邦美債1-3年'],
 ['00694B', '富邦美債1-3年'],
 ['00694B', '富邦美債1-3年'],
 ['00694B', '富邦美債1-3年'],
 ['00695B', '富邦美債7-10年'],
 ['00695B', '富邦美債7-10年'],
 ['00695B', '富邦美債7-10年'],
 ['00695B', '富邦美債7-10年'],
 ['00696B', '富邦美債20年'],
 ['00696B', '富邦美債20年'],
 ['00696B', '富邦美債20年'],
 ['00696B', '富邦美債20年'],
 ['00697B', '元大美債7-10'],
 ['00697B', '元大美債7-10'],
 ['00697B', '元大美債7-10'],
 ['00701', '國泰股利精選30'],
 ['00701', '國泰股利精選30'],
 ['00702', '國泰標普低波高息'],
 ['00702', '國泰標普低波高息'],
 ['00710B', 'FH彭博高收益債'],
 ['00710B', 'FH彭博高收益債'],
 ['00710B', 'FH彭博高收益債'],
 ['00711B', 'FH彭博新興債

In [16]:
df=pd.DataFrame(datas,columns=['編號','股名'])
df

,編號,股名
0,0050,元大台灣50
1,0050,元大台灣50
2,0052,富邦科技
3,0056,元大高股息
4,006203,元大MSCI台灣
...,...,...
1915,9945,潤泰新
1916,9946,三發地產
1917,9951,皇田
1918,9958,世紀鋼


In [1]:
def test(data):
    sqlstr=f'''
    create table if not exists "{data}" (
        id integer primary key autoincrement UNIQUE,
        date text,
        High integer,
        Low integer,
        Open integer,
        Close integer,
        Vol integer
    );
    '''
    try:
        conn=sqlite3.connect('stock.db')
        cursor=conn.cursor()
        conn.execute(sqlstr)
        conn.commit()
        try:
            df=pdr.DataReader(f'{data}.tw','yahoo','2008')
        except Exception as e:
            df=pdr.DataReader(f'{data}.two','yahoo','2008')
        df['date']=df.index.astype(str)
        df1=df[['date','High','Low','Open','Close','Vol']]
        with sqlite3.connect('stock.db') as conn:                   
            df1.to_sql(f'{data}',conn,if_exists='replace',index=False)
    except Exception as e:
        print(e)

In [19]:
df=pd.DataFrame(datas,columns=['編號','股名'])
threads=[]
count=0
now=time.time()
for data in set(df['編號']):
    threads.append(threading.Thread(target=test,args=(data,)))
    print(f'{data}獲取成功')
    count+=1
    if count%25==0:    
        for thread in threads:
            thread.start()
            time.sleep(0.5)     
        for thread in threads:
            thread.join()
        time.sleep(0.5)
        print('Done!')
        threads=[]
for thread in threads:
    thread.start()
    time.sleep(0.5)
for thread in threads:
    thread.join()
print('Done!')   
time.time()-now

6591獲取成功
00722B獲取成功
8049獲取成功
8069獲取成功
3094獲取成功
2887獲取成功
2352獲取成功
6730獲取成功
6143獲取成功
3354獲取成功
1467獲取成功
6655獲取成功
1310獲取成功
5603獲取成功
4555獲取成功
6708獲取成功
3232獲取成功
00724B獲取成功
1470獲取成功
6737獲取成功
2409獲取成功
00771獲取成功
1321獲取成功
2883獲取成功
3564獲取成功
Done!
00781B獲取成功
6721獲取成功
3048獲取成功
3293獲取成功
2323獲取成功
1219獲取成功
3006獲取成功
00869B獲取成功
3390獲取成功
2867獲取成功
1519獲取成功
2101獲取成功
2362獲取成功
2392獲取成功
2417獲取成功
4945獲取成功
2441獲取成功
2480獲取成功
4721獲取成功
1236獲取成功
3306獲取成功
6796獲取成功
8487獲取成功
3058獲取成功
6112獲取成功
Done!
4578獲取成功
4205獲取成功
4431獲取成功
6543獲取成功
8374獲取成功
3708獲取成功
3617獲取成功
6205獲取成功
4716獲取成功
8349A獲取成功
2316獲取成功
1324獲取成功
2915獲取成功
2430獲取成功
5236獲取成功
8119獲取成功
4989獲取成功
2596獲取成功
3206獲取成功
3015獲取成功
3296獲取成功
3332獲取成功
3294獲取成功
3042獲取成功
2211獲取成功
Done!
3529獲取成功
8930獲取成功
3033獲取成功
4556獲取成功
00891獲取成功
1103獲取成功
4767獲取成功
5248獲取成功
5291獲取成功
4766獲取成功
1809獲取成功
8467獲取成功
00888獲取成功
6705獲取成功
8480獲取成功
01001T獲取成功
2640獲取成功
3016獲取成功
00730獲取成功
2072獲取成功
6582獲取成功
4906獲取成功
2520獲取成功
00859B獲取成功
3026獲取成功
Done!
5284獲取成功
5213獲取成功
1731獲取成功
2404獲取成功
6637獲取成功
6185獲取成功
3264獲

Done!
2897獲取成功
6750獲取成功
1309獲取成功
8936獲取成功
5609獲取成功
4164獲取成功
00777B獲取成功
6153獲取成功
2014獲取成功
6416獲取成功
6204獲取成功
00787B獲取成功
2414獲取成功
2753獲取成功
6654獲取成功
6648獲取成功
1260獲取成功
2599獲取成功
6005獲取成功
6212獲取成功
3490獲取成功
3062獲取成功
8072獲取成功
1525獲取成功
6574獲取成功
Done!
8482獲取成功
2030獲取成功
6669獲取成功
6534獲取成功
6606獲取成功
6651獲取成功
2010獲取成功
6664獲取成功
6496獲取成功
2712獲取成功
3211獲取成功
4916獲取成功
6015獲取成功
6257獲取成功
6154獲取成功
8928獲取成功
5285獲取成功
1271獲取成功
2388獲取成功
9940獲取成功
6830獲取成功
2481獲取成功
8016獲取成功
6464獲取成功
8463獲取成功
Done!
4153獲取成功
1735獲取成功
5543獲取成功
6599獲取成功
3231獲取成功
1720獲取成功
4120獲取成功
1315獲取成功
2834獲取成功
2439獲取成功
006203獲取成功
6791獲取成功
2723獲取成功
2881A獲取成功
3002獲取成功
6278獲取成功
4770獲取成功
3005獲取成功
00799B獲取成功
2515獲取成功
8045獲取成功
4575獲取成功
3705獲取成功
3257獲取成功
3675獲取成功
Done!
2916獲取成功
2942獲取成功
00758B獲取成功
1339獲取成功
2230獲取成功
8446獲取成功
6276獲取成功
6612獲取成功
2527獲取成功
6173獲取成功
00701獲取成功
9918獲取成功
01007T獲取成功
00723B獲取成功
4439獲取成功
8289獲取成功
2027獲取成功
6797獲取成功
5534獲取成功
5520獲取成功
8924獲取成功
00882獲取成功
2451獲取成功
3017獲取成功
1232獲取成功
Done!
2415獲取成功
2608獲取成功
9929獲取成功
5243獲取成功
3260獲取成功
00788B獲取

1100.7490453720093

In [20]:
def deposit():
    sqlstr='''
    create table if not exists "deposit" (
        id integer primary key autoincrement UNIQUE,
        "編號" text,
        "股名" text,
        "除息日" text,
        "現金股利" text,
        "除息參考價" text,
        "現金殖利率" text
    );
    '''
    get_chrom(url)
    xpath='/html/body/table[2]/tbody/tr/td[1]/div[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td/nobr/a'
    element=chrom.find_element(By.XPATH,xpath)
    element.click()
    count=0
    page=4
    datas=[]
    while count<1:
        try:
            xpath=f'/html/body/table[2]/tbody/tr/td[3]/table/tbody/tr/td/div/form/nobr[3]/select/option[{page}]'
            chrom.find_element(By.XPATH,xpath).click()
            soup=BeautifulSoup(chrom.page_source,'lxml')
            test=soup.find('div',style="position:relative;width:811px;").find('tbody')
            for i in test.find_all('tr',align="center"):
                day=i.find_all('td')[4].text
                money=i.find('td',align="right").text
                price=i.find_all('td')[5].text
                if price !='':
                    yield_=round((eval(money)/eval(price))*100,2)
                else:
                    yield_=0
                datas.append([i.find('a',class_="link_black").text,i.find_all('td')[2].text,day,money,price,yield_])
            page+=1
        except Exception as e:
            count+=1
    df=pd.DataFrame(datas,columns=['編號','股名','除息日','現金股利','除息參考價','現金殖利率'])
    conn=sqlite3.connect('stock.db')
    cursor=conn.cursor()
    conn.execute(sqlstr)
    conn.commit()
    with sqlite3.connect('stock.db') as conn:                   
        df.to_sql('deposit',conn,if_exists='replace',index=False)

In [21]:
deposit()